In [1]:
%matplotlib inline
import sys
sys.path.insert(0, '../')
import os
os.chdir('..')

import tensorflow as tf
import numpy as np
import librosa
from utils.strechableNumpyArray import StrechableNumpyArray
import matplotlib.pyplot as plt
import IPython
import functools
from tensorflow.contrib.signal.python.ops import window_ops

In [2]:
tf.reset_default_graph()

import pickle

from architecture.contextEncoderArchitecture import ContextEncoderArchitecture
from system.contextEncoderSystem import ContextEncoderSystem
from system.preAndPostProcessor import PreAndPostProcessor

architecturesParametersFile = "complex_network_parameters.pkl"
sessionsName = "Papers_FMA_Context_Encoder"

with open(architecturesParametersFile, 'rb') as savedFile:
    Context_Encoder_parameters = pickle.load(savedFile)

aContextEncoderArchitecture = ContextEncoderArchitecture(*Context_Encoder_parameters.architectureParameters())
aPreProcessor = PreAndPostProcessor(*Context_Encoder_parameters.preProcessorParameters())
aContextEncoderSystem = ContextEncoderSystem(aContextEncoderArchitecture, Context_Encoder_parameters.batchSize(),
                                             aPreProcessor, sessionsName)


---------
Encoder
---------
Tensor("ContextEncoderArchitecture/input_data:0", shape=(256, 16, 257, 4), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Conv_0/Conv2D:0", shape=(256, 8, 129, 32), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Relu:0", shape=(256, 8, 129, 32), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/batch_normalization/cond/Merge:0", shape=(256, 8, 129, 32), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Conv_1/Conv2D:0", shape=(256, 4, 43, 128), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Relu_1:0", shape=(256, 4, 43, 128), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/batch_normalization_2/cond/Merge:0", shape=(256, 4, 43, 128), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Conv_2/Conv2D:0", shape=(256, 2, 15, 512), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Relu_2:0", shape=(256, 2, 15, 512), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/batch_normalizatio

In [3]:
sr = 16000
start_in_seconds = 0.1
gap_length = 1024
side_length = 2048

starting_sample_left_side = int(sr*start_in_seconds)
ending_sample_left_side = starting_sample_left_side + side_length
starting_sample_right_side = ending_sample_left_side + gap_length
ending_sample_right_side = starting_sample_right_side + side_length

best_step = 2567124 # trained only on FMA


batch_size = 256

In [5]:
def _pavlovs_SNR(y_orig, y_inp, onAxis=(1,)):
    norm_y_orig = _squaredEuclideanNorm(y_orig, onAxis)
    norm_y_orig_minus_y_inp = _squaredEuclideanNorm(y_orig - y_inp, onAxis)
    return 10 * np.log10(norm_y_orig / norm_y_orig_minus_y_inp)

def _squaredEuclideanNorm(vector, onAxis=(1,)):
    squared = np.square(vector)
    summed = np.sum(squared, axis=onAxis)
    return summed

def computeInpaintingSNRFor(signals_batch):
    original_signals = signals_batch[:, 5120:5120*2]
    original_gaps = signals_batch[:, 5120+2048:5120+2048+1024]
    generatedSpecs = aContextEncoderSystem.reconstructAudio(original_signals, model_num=best_step)
    complexGenerated = generatedSpecs[0][:, 3:-3, :, 0] + 1.0j * generatedSpecs[0][:, 3:-3, :, 1]
    complexOriginal = generatedSpecs[1][:, 3:-3, :, 0] + 1.0j * generatedSpecs[1][:, 3:-3, :, 1]

    SNRs = _pavlovs_SNR(np.abs(complexOriginal), np.abs(complexGenerated), onAxis=(1, 2))
    SNRs[np.isinf(SNRs)] = 0
    return SNRs



In [6]:
import scipy
train = scipy.io.loadmat('FMA_train_windows_16k.mat')
print(train['fma_test'].shape)
trainSNRs = computeInpaintingSNRFor(train['fma_test'])

valid = scipy.io.loadmat('FMA_valid_windows_16k.mat')
print(valid['fma_test'].shape)
validSNRs = computeInpaintingSNRFor(valid['fma_test'])

test = scipy.io.loadmat('FMA_test_windows_16k.mat')
print(test['fma_test'].shape)
testSNRs = computeInpaintingSNRFor(test['fma_test'])

print('train mean SNR:', np.mean(trainSNRs))
print('train std SNR:', np.std(trainSNRs))
print('valid mean SNR:', np.mean(validSNRs))
print('valid std SNR:', np.std(validSNRs))
print('test mean SNR:', np.mean(testSNRs))
print('test std SNR:', np.std(testSNRs))


(4002, 32000)
INFO:tensorflow:Restoring parameters from utils/saved_models/Papers_FMA_Context_Encoder/model-Papers_FMA_Context_Encoder2567124.ckpt
Model restored.


/home/amarafioti/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.


(4000, 32000)
INFO:tensorflow:Restoring parameters from utils/saved_models/Papers_FMA_Context_Encoder/model-Papers_FMA_Context_Encoder2567124.ckpt
Model restored.
(3547, 32000)
INFO:tensorflow:Restoring parameters from utils/saved_models/Papers_FMA_Context_Encoder/model-Papers_FMA_Context_Encoder2567124.ckpt
Model restored.
train mean SNR: 4.9019456
train std SNR: 4.0436177
valid mean SNR: 5.14429
valid std SNR: 4.1795306
test mean SNR: 5.4540076
test std SNR: 4.4836307


In [9]:
tf.reset_default_graph()

sessionsName = "Papers_Context_Encoder"

with open(architecturesParametersFile, 'rb') as savedFile:
    Context_Encoder_parameters = pickle.load(savedFile)

aContextEncoderArchitecture = ContextEncoderArchitecture(*Context_Encoder_parameters.architectureParameters())
aPreProcessor = PreAndPostProcessor(*Context_Encoder_parameters.preProcessorParameters())
aContextEncoderSystem = ContextEncoderSystem(aContextEncoderArchitecture, Context_Encoder_parameters.batchSize(),
                                             aPreProcessor, sessionsName)

best_step = 967467


---------
Encoder
---------
Tensor("ContextEncoderArchitecture/input_data:0", shape=(256, 16, 257, 4), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Conv_0/Conv2D:0", shape=(256, 8, 129, 32), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Relu:0", shape=(256, 8, 129, 32), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/batch_normalization/cond/Merge:0", shape=(256, 8, 129, 32), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Conv_1/Conv2D:0", shape=(256, 4, 43, 128), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Relu_1:0", shape=(256, 4, 43, 128), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/batch_normalization_2/cond/Merge:0", shape=(256, 4, 43, 128), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Conv_2/Conv2D:0", shape=(256, 2, 15, 512), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/Relu_2:0", shape=(256, 2, 15, 512), dtype=float32)
Tensor("ContextEncoderArchitecture/Encoder/batch_normalizatio

In [10]:
train = scipy.io.loadmat('NSynth_train_windows_16k.mat')
print(train['nsynth_test'].shape)
trainSNRs = computeInpaintingSNRFor(train['nsynth_test'])

valid = scipy.io.loadmat('NSynth_valid_windows_16k.mat')
print(valid['nsynth_test'].shape)
validSNRs = computeInpaintingSNRFor(valid['nsynth_test'])

test = scipy.io.loadmat('NSynth_test_windows_16k.mat')
print(test['nsynth_test'].shape)
testSNRs = computeInpaintingSNRFor(test['nsynth_test'])

print('train mean SNR:', np.mean(trainSNRs))
print('train std SNR:', np.std(trainSNRs))
print('valid mean SNR:', np.mean(validSNRs))
print('valid std SNR:', np.std(validSNRs))
print('test mean SNR:', np.mean(testSNRs))
print('test std SNR:', np.std(testSNRs))




(3894, 32000)
INFO:tensorflow:Restoring parameters from utils/saved_models/Papers_Context_Encoder/model-Papers_Context_Encoder967467.ckpt
Model restored.
(3886, 32000)
INFO:tensorflow:Restoring parameters from utils/saved_models/Papers_Context_Encoder/model-Papers_Context_Encoder967467.ckpt
Model restored.
(3984, 32000)
INFO:tensorflow:Restoring parameters from utils/saved_models/Papers_Context_Encoder/model-Papers_Context_Encoder967467.ckpt
Model restored.
train mean SNR: 17.812756
train std SNR: 10.517775
valid mean SNR: 18.270575
valid std SNR: 10.320516
test mean SNR: 18.222937
test std SNR: 10.133304
